# Writing Programs for Aquila

## Aquila Analog Computer

The Aquila system will evolve neutral atoms under the following interacting Rydberg Hamiltonian:

$$
\mathcal{H}(t) = \mathcal{H}_{\textrm{global}}(t) + \mathcal{H}_{\textrm{detuning}}(t) + \mathcal{H}_{\textrm{interaction}}(t)
$$

$$
\frac{\mathcal{H}_{\textrm{global}}(t)}{\hbar} = \sum_j \frac{\Omega_j(t)}{2} \bigl(e^{i\phi_j(t)} \ket{g_j(t)} \bra{r_j(t)}+e^{-i\phi_j(t)} \ket{r_j(t)} \bra{g_j(t)}\bigr) 
$$
$$
\frac{\mathcal{H}_{\textrm{detuning}}(t)}{\hbar} = - \sum_j \Delta_j(t) \hat{n_j}
$$
$$
\frac{\mathcal{H}_{\textrm{interaction}}(t)}{\hbar} = \sum_{<j} V_{j,k} \hat{n_j}\hat{n_k}, \textrm{where} V_{j,k}=C_6/∣x_j−x_k∣^6
$$

Thus, when programming a system, it is important to note that the three terms each play a role and have different associated noise. 


## Query Aquila capabilities 
To see the docstring in a Jupyter notebook, type out the object you're interested in and press `shift + tab`.

The `bloqade function` `get_capabilities` returns the parameter value limits for Aquillia. 

We can use `shift + tab` to see its docstring, which defines the units for the returned values.

In [2]:
import pprint # A standard Python library for 'pretty printing'
from bloqade import get_capabilities 

capabilities = get_capabilities().dict()['capabilities']
pprint.pprint(capabilities)

{'lattice': {'area': {'height': Decimal('76'), 'width': Decimal('75')},
             'geometry': {'number_sites_max': 256,
                          'position_resolution': Decimal('0.1'),
                          'spacing_radial_min': Decimal('4'),
                          'spacing_vertical_min': Decimal('4')},
             'number_qubits_max': 256},
 'rydberg': {'c6_coefficient': Decimal('5.42E+6'),
             'global_': {'detuning_max': Decimal('125.0000000'),
                         'detuning_min': Decimal('-125.0000000'),
                         'detuning_resolution': Decimal('2E-7'),
                         'detuning_slew_rate_max': Decimal('2500.0000000000000'),
                         'phase_max': Decimal('99.0'),
                         'phase_min': Decimal('-99.0'),
                         'phase_resolution': Decimal('5E-7'),
                         'rabi_frequency_max': Decimal('15.8000000'),
                         'rabi_frequency_min': Decimal('0E-7'),
         

In [5]:
# this cell is an example of what you would see with shift + tab
# great way to check the units 
get_capabilities

<function bloqade.factory.get_capabilities(use_experimental: bool = False) -> 'QuEraCapabilities'>

## Running some bloqade geometries

To write a program for Aquila, we start with an atom geometry. Now lets try running some setups. Let's create a Chain of 4 atoms that are each $50 \, \mu m$ apart. Take a look at its methods and attributes.

**Note**: Geometry is actually important on the real system because certain geometries are less noise than others (for example vertical rather than horizontal.)

*Remember*: to see the methods and attributes of an object instance, first instantiate the object by evaluating the cell (`shift + return`), then type `.` followed by `tab`.


In [6]:
from math import pi
from bloqade.atom_arrangement import Chain

In [10]:
geometry = Chain(4, lattice_spacing = 50.0)
# you can always look at the geometry visually.
geometry

                                   Atom Positions                               
     ┌─────────────────────────────────────────────────────────────────────────┐
 1.00┤                                                                         │
     │                                                                         │
     │                                                                         │
 0.67┤                                                                         │
     │                                                                         │
     │                                                                         │
 0.33┤                                                                         │
     │                                                                         │
     │                                                                         │
 0.00┤•                       •                       •                       •│
     │                      

Methods to create pulse sequences that manipulate the rabi pulse or local detunings are found under `geometry.rydberg`.

In [11]:
# Again, try a shift+tab here
geometry.rydberg

## Evolving the system 

Let's create a simple program that applies a NOT gate to the initial $\ket{1111}$ state, by applying a $\pi$ Rabi pulse (a $\pi$ rotation on the Bloch sphere). 

*Remember*: 
* Atoms are initialised in the $\ket{1}$ state. 
* All atoms (qubits) undergo this evolution so you should get $\ket{1111}\rightarrow\ket{0000}$. 

In [12]:
NOT = (
  geometry
  .rydberg.rabi.amplitude.uniform
  .constant(pi,1)
)

We can simulate the program with the Python backend as follows.

**NOTE**: $10000$ shots would cost USD $~1300$! (Also, is it even within the device capability limits?)

In [13]:
n_shots = 10000
results = NOT.bloqade.python().run(n_shots)
results.report().counts()

[OrderedDict([('0000', 10000)])]

The `results` object contains quite a bit of information, use `results.report().show()` to open a visualisation of the simulation results.
**Note** that the above results are for an *ideal* system and setup. 

In [18]:
# this can be a bit temperamental and also will open a window in the default browser
results.report().show()

0.0 150000000.0 0.0 0.0


To run a simulation on Aquilla the atom geometry and pulse sequence must be within the its capability limits.

With that in mind, there are a couple issues with our `NOT` program:

* The chain has a horizontal length of $200 \, \mu m$, while the maximum width is $75 \, \mu m$.
* A `constant` pulse is not possible in practice; the Rabi amplitude waveform must start and end at zero.

In [19]:
# Reducing the atom spacing.

geometry = Chain(4, lattice_spacing = 15.0)

NOT = (
  geometry
  .rydberg.rabi.amplitude.uniform
  .constant(pi,1)
)

results = NOT.bloqade.python().run(n_shots)
results.report().counts()

[OrderedDict([('0000', 9302),
              ('0100', 247),
              ('0010', 231),
              ('1000', 64),
              ('0001', 47),
              ('0110', 45),
              ('1100', 29),
              ('0011', 28),
              ('1010', 3),
              ('0101', 2),
              ('1101', 1),
              ('1110', 1)])]

Now instead the results show a large number of *incorrect* answers but the correct result accounts for $93\%$. Yet this is not representative of what could be run on the system. We need to adjust the pulse. Instead of `uniform` we need to specify a continuous pulse sequence. Here we use `peicewise_linear`.

In [20]:
NOT = (
  geometry
  .rydberg.rabi.amplitude.uniform
  .piecewise_linear(values = [0, pi, pi, 0], durations = [0.06, 1, 0.06])
)

results = NOT.bloqade.python().run(n_shots)
results.report().counts()

[OrderedDict([('0000', 8972),
              ('0100', 287),
              ('0010', 262),
              ('0001', 140),
              ('1000', 115),
              ('0110', 87),
              ('0011', 62),
              ('1100', 60),
              ('0101', 4),
              ('1010', 3),
              ('1011', 3),
              ('1110', 3),
              ('0111', 2)])]

Again, the correct result is $<100\%$ and has even gotten worse. 

As the pulse now has a ramp-up and ramp-dow period, we need to factor this into the total pulse area.

By reducing the time spent at a Rabi amplitude of $\pi$ based on the area of the ramp-up and ramp-down periods, the accuracy of the `NOT` program improves.

In [15]:
area_correction = 2*0.06*0.5

NOT = (
  geometry
  .rydberg.rabi.amplitude.uniform
  .piecewise_linear(values = [0, pi, pi, 0], durations = [0.06, 1 - area_correction, 0.06])
)
# if you want to see the program to be run 
# NOT.show()

results = NOT.bloqade.python().run(n_shots)
results.report().counts()

[OrderedDict([('11', 3648), ('10', 3194), ('01', 3158)])]

Now we are back to $93\%$. Yet this is for lots of shots AND also assumes an ideal system. There is no noise here. This is a key thing to consider. Now consider trying this using a job submission script and compare the results.

## Submitting 
Now let's consider submitting to Aquila itself. This relies on make using of a magic token and a url. The function below takes a program (sequence of pulses applied to a geometry) and sends this to the url for Aquila with the appropriate key.


In [37]:
def SubmitToAquila(program, 
                   nshots, 
                   magic_key = "fake_key", 
                   url = "fake_url", 
                   body_template = "{task_ir}"
                  ):
    """Submit a job to Aquila and check that response code indicates it has been submitted
    """
    
    if magic_key == "fake_key" or url == "fake_url":
        print(f"Not submitting program ... given magic_key={magic_key}, url={url}")
        return 
    request_options = dict(params={"key": magic_key})
    result = program.quera.custom().submit(nshots, url, body_template, request_options = request_options)

    # Response code of 200 if the submission was successful.
    while result != "200":
        sleep(1)


In [48]:
geometry = Chain(2, lattice_spacing = 6)
ramp_time=0.06
area_correction = 2*ramp_time*0.5
pulse_sequence = start.rydberg.rabi.amplitude.uniform.piecewise_linear(
    values = [0, pi, pi, 0], 
    durations = [ramp_time, 1 - area_correction, ramp_time]
)
pulse_sequence = pulse_sequence.parse_sequence()
program = geometry.apply(pulse_sequence)

# could set the pulse for the geometry directly but does mean can't build a series of local detunings easily
# program = geometry.rydberg.rabi.amplitude.uniform.piecewise_linear(
#     values = [0, pi, pi, 0], 
#     durations = [ramp_time, 1 - area_correction, ramp_time]
# )

SubmitToAquila(program, 100)


Not submitting program ... given magic_key=fake_key, url=fake_url


## CNOT?
This set of instructions showcases adding local detuning along with the global pulse. The goal is to see if we can construct a CNOT gate using a global pulse, local detuning and the geometry of the system, a $\ket{11}$. 

The global pulse here consists of 3 $\pi$ pulses, where we ramp and then down. Here $\mathcal{H}\ket{\Phi}$ is equivalent to 3 NOT pulses, ie: $\ket{11}\rightarrow\ket{00}$. At least that would be the case in the absence of an interaction term (try placing with the spacing). With the interaction term, you get a mix of $\ket{11}, \ket{01}, \ket{10}$ states. 

The local detuning then ensures that instead the second qubit's value is dictated by the first giving mostly $\ket{11}$. 

In [80]:
# lets setup the two qubits 

# if verbose is set to true, show the programs and the results as full windows.
verbose = False

# if want to try seeing how the interaction term affects system
NOT_spacing = False
spacing = 6
if NOT_spacing == True:
    spacing = 40.0 
    
geometry = Chain(2, lattice_spacing = spacing)

# number of shots
nshots = 100

# a global pulse
pulse_sequence = (start
    .rydberg
    .rabi
    .amplitude
    .uniform
    .piecewise_linear([0.06, 1, 0.06, 0.06, 1, 0.06, 0.06, 1, 0.06],[0, pi, pi, 0, pi, pi, 0, pi, pi, 0]))

# lets look at what the global pulse would do on its own
global_pulse_sequence = pulse_sequence.parse_sequence()
global_pulse_program = geometry.apply(global_pulse_sequence)
results = global_pulse_program.bloqade.python().run(nshots)
if verbose == True:
    global_pulse_program.show()
    results.report().show()
bitstring_counts = results.report().counts()
print("Global pulse results", bitstring_counts)

# local detuning pulses 
# add pulse to first qubit
pulse_sequence = (pulse_sequence.rydberg
    .detuning
    .location(0)
    .piecewise_linear([0.06, 1, 0.06, 0.06, 1, 0.06, 0.06, 1, 0.06], [0, -50, -50, 0, 0, 0, 0, -50, -50, 0]))

# and then second qubit
pulse_sequence = (pulse_sequence.rydberg
    .detuning
    .location(1)
    .piecewise_linear([1.12, 0.06, 1, 0.06],[0, 0, -50, -50, 0]))

# setup the system
pulse_sequence = pulse_sequence.parse_sequence()
geometry = Chain(2, lattice_spacing = spacing)
program = geometry.apply(pulse_sequence)

# if you want to see program uncomment
results = program.bloqade.python().run(nshots)
if verbose == True:
    program.show()
    results.report().show()
bitstring_counts = results.report().counts()
print("Local detuning added", bitstring_counts)


Global pulse results [OrderedDict([('11', 49), ('10', 26), ('01', 25)])]
Local detuning added [OrderedDict([('11', 98), ('01', 1), ('10', 1)])]


## Other Geometries 
Here we show some other geometries. The best place to get some information is [https://bloqade.quera.com/latest/home/quick_start/#defining-atom-geometry]

In [81]:
from bloqade.atom_arrangement import Square, Kagome
simple_geometry = Square(2, 4, lattice_spacing = 4.0)
more_complex_geometry = Kagome(2, 2, lattice_spacing = 2.0)
even_more_complex_geometry = defective_geometry = more_complex_geometry.apply_defect_density(0.2)

# and now can construt a specific geometry
my_geometry = start.add_position([(1,2), (3,4), (5,6)])
my_geometry_odd_qubit_order = start.add_position([(5,23.0), (0.,0.), (74.0,74.0), (55.55,2.0)])

In [57]:
simple_geometry 

                                  Atom Positions                                
  ┌────────────────────────────────────────────────────────────────────────────┐
12┤•                                                                          •│
  │                                                                            │
  │                                                                            │
10┤                                                                            │
  │                                                                            │
  │                                                                            │
 8┤•                                                                          •│
  │                                                                            │
  │                                                                            │
 6┤                                                                            │
  │                         

In [58]:
more_complex_geometry

                                   Atom Positions                               
    ┌──────────────────────────────────────────────────────────────────────────┐
2.60┤                           •                                    •         │
    │                                                                          │
    │                                                                          │
2.17┤                                                                          │
    │                                                                          │
    │                                                                          │
1.73┤                  •                  •                 •                 •│
    │                                                                          │
    │                                                                          │
1.30┤                                                                          │
    │                       

In [64]:
my_geometry

                                   Atom Positions                               
    ┌──────────────────────────────────────────────────────────────────────────┐
6.00┤                                                                         •│
    │                                                                          │
    │                                                                          │
5.33┤                                                                          │
    │                                                                          │
    │                                                                          │
4.67┤                                                                          │
    │                                                                          │
    │                                                                          │
4.00┤                                     •                                    │
    │                       

In [65]:
my_geometry_odd_qubit_order

                                   Atom Positions                               
    ┌──────────────────────────────────────────────────────────────────────────┐
74.0┤                                                                         •│
    │                                                                          │
    │                                                                          │
61.7┤                                                                          │
    │                                                                          │
    │                                                                          │
49.3┤                                                                          │
    │                                                                          │
    │                                                                          │
37.0┤                                                                          │
    │                       